# Library importing

In [1]:
!pip install xgboost
!pip install --upgrade install
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/115.9 MB ? eta -:--:--

   ━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/115.9 MB 224.0 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 89.1/115.9 MB 252.5 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 115.9/115.9 MB 257.1 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 115.9/115.9 MB 257.1 MB/s eta 0:00:01

...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 296.7/296.8 MB 270.8 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 34.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


ERROR: Could not find a version that satisfies the requirement install (from versions: none)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for install


# data pre-proccessing

## data reading

In [2]:
final_data = pd.read_csv("/kaggle/input/data-cleaning/preprocessing")
final_data.drop(columns='Unnamed: 0',inplace =True)

In [3]:
final_data.head(3)

,item_id,store_id,sales,wm_yr_wk,wday,event_name_1,event_type_1,event_name_2,event_type_2,snap,year,month,day,price
0,FOODS_1_001,CA_1,3,1,1,No event,No event,No event,No event,0,2011,1,29,2.0
1,FOODS_1_001,CA_1,0,2,3,No event,No event,No event,No event,1,2011,2,7,0.0
2,FOODS_1_001,CA_1,0,15,2,Mother's day,Cultural,No event,No event,1,2011,5,8,0.0


## Encoding

In [4]:
categorical_cols = ['item_id','store_id','event_name_1','event_name_2','event_type_1','event_type_2']
numeric_cols = ["wm_yr_wk", "wday", "snap","year", "month", "day", "price"]

In [5]:
categorical_transformer = Pipeline(steps=[
    ("encoder", OrdinalEncoder()),
    ("scaler", MinMaxScaler())
])
numeric_transformer = "passthrough"
preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_transformer, categorical_cols),
    ("num", numeric_transformer, numeric_cols)
])

In [6]:
preprocessor

,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'


In [7]:
joblib.dump(preprocessor,'preprocessing_pipeline.pkl')

['preprocessing_pipeline.pkl']

## Data spliting

In [8]:
y=final_data['sales']
x=final_data.drop('sales',axis = 1)

# Model

In [9]:
x=preprocessor.fit_transform(x)

In [10]:
model = XGBRegressor(
    bjective = 'reg:squarederror',
    booster =  "gbtree",
    n_estimator = 500 ,
    max_depth = 20 ,
    random_state = 42 ,
    learning_rate = 0.01 ,
    eval_metric = 'logloss',
    early_stopping_rounds = 10
)

In [11]:
x_temp , x_test , y_temp , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 42)
x_train , x_val , y_train , y_val = train_test_split(x_temp , y_temp , test_size = 0.15 , random_state = 42)

In [12]:
model.fit(x_train,y_train,eval_set = [(x_val , y_val)])

/usr/local/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [11:55:16] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "bjective", "n_estimator" } are not used.

  self.starting_round = model.num_boosted_rounds()


[0]	validation_0-logloss:-4.79757


[1]	validation_0-logloss:-4.85978


[2]	validation_0-logloss:-4.91369


[3]	validation_0-logloss:-4.96180


...

[96]	validation_0-logloss:-30.84228


[97]	validation_0-logloss:-30.85159


[98]	validation_0-logloss:-30.86068


[99]	validation_0-logloss:-30.86965


,objective,'reg:squarederror'
,base_score,None
,booster,'gbtree'
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,10
,enable_categorical,False
,eval_metric,'logloss'


In [13]:
model.score(x_train,y_train) 

0.6329197883605957

In [14]:
model.score(x_test,y_test)

0.5742210745811462